In [ ]:
import pandas as pd

In [ ]:
def getHeatmapData(df, colname, step, outfile):
    min = df[colname].min()
    max = df[colname].max()
    quant = []
    for i in range(step):
        quant.append(i * (1 / step) * max)
    quant.append(max)
    df.groupby(['sec', pd.cut(df[colname], quant)]).size().to_csv(outfile)

In [ ]:
def getHeatmapDataAllStep(df, colname, outfile):
    min = df[colname].min()
    max = df[colname].max()
    quant = range(0, 33)
    
    df.groupby(['sec', pd.cut(df[colname], quant)]).size().to_csv(outfile)

In [ ]:
inputFile = './parsedData/trace_scsi.txt.json.calc.csv'
df = pd.read_csv(inputFile)
df['sec'] = df['issue_time'].astype(int)
print(df.dtypes)

In [ ]:
file_lba = './parsedData/heatmap_lba.csv'
file_queue = './parsedData/heatmap_queue.csv'
file_latency = './parsedData/heatmap_latency.csv'


# getHeatmapData(df, 'lba', 20, file_lba)
# getHeatmapData(df, 'latency(us)', 20, file_latency)
# getHeatmapDataAllStep(df, 'queue_cnt', file_queue)

# 위 함수 수행 후, y축 값 엑셀에서 손으로 변경하고, tab -> comma 찾아 바꾸기 해야함
